# Проект по SQL 

### Описание проекта

**Цель исследования** - получить данные для формулировки ценностного предложения для крупного сервиса для чтения книг по подписке.

**Задача** - проанализировать базу данных (информация о книгах, издательствах, авторах, а также пользовательские обзоры книг).

**Шаги для анализа базы данных:**

- Посчитать, сколько книг вышло после 1 января 2000 года;

- Для каждой книги посчитать количество обзоров и среднюю оценку;

- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;

- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Загрузка данных и изучение общей информации

In [2]:
# таблица с данными о книгах
query = ''' SELECT *
        FROM books ''' 

# выводим результат
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


<div class="alert alert-info">
    
**📝 Комментарий Маши**
    
База содержит информацию о 1000 книгах (идентификатор, название, кол-во страниц,дата публикации и идентификатор издательства).
    
</div>

In [3]:
# таблица с данными об авторах
query = ''' SELECT *
        FROM authors ''' 

# выводим результат
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


<div class="alert alert-info">
    
**📝 Комментарий Маши**
    
База содержит информацию о 636 авторах (их идентификатор и имя).
    
</div>

In [4]:
# таблица с данными об издательствах
query = ''' SELECT *
        FROM publishers ''' 

# выводим результат
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


<div class="alert alert-info">
    
**📝 Комментарий Маши**
    
База содержит информацию о 340 издательства (их идентификатор и название).
    
</div>

In [5]:
# таблица с данными о пользовательских оценках книг
query = ''' SELECT *
        FROM ratings ''' 

# выводим результат
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


<div class="alert alert-info">
    
**📝 Комментарий Маши**
    
База содержит информацию о 3456 пользовательскх оценках книг (идентификатор оценки, идентификатор книги, имя пользователя и оценка книги).
    
</div>

In [6]:
# таблица с данными о пользовательских обзорах на книги
query = ''' SELECT *
        FROM reviews ''' 

# выводим результат
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


<div class="alert alert-info">
    
**📝 Комментарий Маши**
    
База содержит информацию о 2793 пользовательских обзорах на книги (идентификатор обзора, идентификатор книги, имя пользователя и текст обзора).
    
</div>

## Исследовательский анализ данных

### Посчитать, сколько книг вышло после 1 января 2000 года;

In [7]:
# формулировка запроса
query = ''' SELECT COUNT(book_id)
        FROM books
        WHERE publication_date > '2000-01-01'  '''

# выводим результат
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


<div class="alert alert-info">
    
**📝 Комментарий Маши**
    
В базе 819 книг, которые вышли после 1 января 2000 года. 
    
</div>

### Для каждой книги посчитать количество обзоров и среднюю оценку;

In [8]:
# формулировка запроса
query = ''' SELECT b.title,
                   COUNT(DISTINCT r.review_id) AS review_count,
                   AVG(rat.rating) AS avg_rat
            FROM books AS b
            LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
            LEFT JOIN reviews AS r ON b.book_id = r.book_id
            GROUP BY b.book_id
            ORDER BY review_count DESC'''

# выводим результат
pd.io.sql.read_sql(query, con = engine) 

,title,review_count,avg_rat
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


<div class="alert alert-info">
    
**📝 Комментарий Маши**
    
Для каждой книги посчитаны количество обзоров и средняя оценка. Таблица отсортирована по количеству отзывов. Больше всего отзывов у Twilight (Twilight #1) (7).
    
</div>

### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

In [9]:
# формулировка запроса
query = ''' SELECT p.publisher,
                   COUNT(book_id)
            FROM books AS b
            JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY p.publisher
            ORDER BY COUNT(book_id) DESC
            LIMIT 1 '''

# выводим результат
pd.io.sql.read_sql(query, con = engine) 

,publisher,count
0,Penguin Books,42


<div class="alert alert-info">
    
**📝 Комментарий Маши**
    
Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books	(42 книги).
    
</div>

### Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;

In [10]:
# формулировка запроса
query = ''' WITH rat AS (SELECT book_id,
                                AVG(rating) AS avg_rating,
                                COUNT(rating_id) AS count_rating
                         FROM ratings
                         GROUP BY book_id)
            SELECT a.author,
                   AVG(rat.avg_rating) AS avg 
            FROM books AS b
            JOIN rat ON b.book_id = rat.book_id
            JOIN authors AS a ON b.author_id = a.author_id
            WHERE rat.count_rating >= 50
            GROUP BY a.author
            ORDER BY avg DESC
            LIMIT 1 '''

pd.io.sql.read_sql(query, con = engine) # выводим результат

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


<div class="alert alert-info">
    
**📝 Комментарий Маши**
    
Автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré	 (учитывались книги с 50 и более оценками).
    
</div>

### Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [11]:
# формулировка запроса
query = ''' WITH rat AS (SELECT username,
                                COUNT(rating_id) AS count_rating
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating_id) > 50), 
            rev AS (SELECT username,
                           COUNT(review_id) AS count_review
                    FROM reviews
                    GROUP BY username)
            SELECT ROUND(AVG(count_review)) as avg_review
            FROM rat
            JOIN rev ON rat.username = rev.username '''

# выводим результат
pd.io.sql.read_sql(query, con = engine) 

,avg_review
0,24.0


<div class="alert alert-info">
    
**📝 Комментарий Маши**
    
Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24. 
    
</div>

## Общий вывод

Анализ базы данных крупного сервиса для чтения книг по подписке показал, что:
- В базе 819 книг, которые вышли после 1 января 2000 года.
- Для каждой книги посчитаны количество обзоров и средняя оценка. Таблица отсортирована по количеству отзывов. Больше всего отзывов у Twilight (Twilight #1) (7).
- Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books (42 книги).
- Автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré (учитывались книги с 50 и более оценками).
- Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24. 